In [ ]:
import sys
sys.path.insert(1, '../')
import analyze_kinetics
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## Heatmap validation

### In order to run the code below, you need to generate "sorption_fitting_results_27C.json" file by running "Figures_S41-S97, S99-S101+calculate_heatmaps.ipynb" notebook

In [ ]:
import json

with open('sorption_fitting_results_27C.json', 'r') as infile:
    data = json.load(infile)
    print(data)

In [ ]:
title_dictionary = {'0': 'Desorption', '1':'Adsorption 30 %RH', '2':'Adsorption 60 %RH',}
material_list = [ 'ROS-037', 'ROS-039', 'ROS-040', 'MOF-303', 'MIL-160', 'Syloid', 'CAU-10', 'Al-fumarate']
for RH_target in [ '0', '1',  '2']:
    plt.bar(material_list, 
        [data[material][1]['k'][RH_target] for material in material_list], 
        yerr=[data[material][1]['k_sigma'][RH_target] for material in material_list], 
        align='center', alpha=0.5, ecolor='black', capsize=10)
    plt.title(title_dictionary[RH_target])
    plt.ylabel('k\', (wt.%·mg)/(min·%RH)')
    plt.xticks(rotation=90)
    plt.show()

In [ ]:
  
if True:
    path_dictionary = {'ROS-037':'../../data/ROS-037/27C/hm/',
                       'MIL-160':'../../data/MIL-160/27C/hm/',
                       'MOF-303':'../../data/MOF-303/27C/hm/',
                        'ROS-039':'../../data/ROS-039/27C/hm/',
                       'ROS-040':'../../data/ROS-040/27C/hm/',
                       'Syloid':'../../data/Syloid/27C/hm/',
                       }
    for material in path_dictionary.keys():
        def curve( w0,  K_ads, K_des, t_range, RH):
            uptake_out = []
            B = w0
            t0 = t_range[0]
            for i, t in enumerate(t_range):
                B = B + (t-t0)* (K_ads*((RH[i] - sorption_list[material]['isotherm'].uptake_to_RH_ads(B))>0) 
                                 + K_des*((RH[i] - sorption_list[material]['isotherm'].uptake_to_RH_ads(B))<=0) )* \
                                (RH[i] - sorption_list[material]['isotherm'].uptake_to_RH_ads(B))   
                uptake_out.append(B)
                t0=t
            #print(t_range, RH)
            #plt.plot(t_range, uptake_out)
            #plt.show()
            return uptake_out
        
        plt.plot(np.linspace(min(sorption_list[material]['isotherm'].adsorption), max(sorption_list[material]['isotherm'].adsorption), 100), 
                 sorption_list[material]['isotherm'].uptake_to_RH_ads(np.linspace(min(sorption_list[material]['isotherm'].adsorption), max(sorption_list[material]['isotherm'].adsorption), 100)))
        plt.show()
        
        path = path_dictionary[material]
        filelist = os.listdir(path)
        filelist = [i for i in filelist if (i[-5:] == '.xlsx')|(i[-4:]=='.xls')]
        sorption_list[material]['HM_validation'] = {}
       
    
        print(material)
        print(filelist)
        for val_nr, filename in enumerate(filelist):
            f = analyze_kinetics.Sorption()
            f.read_file(path, filename)
            print(f.filename)
            print(f.experiment_type)
            sorption_list[material]['data'].append(f)
            if f.experiment_type != 'isotherm':
                #sorption_list['ROS-037']['kinetics'] = kin
                #sorption_list['ROS-037'].append(f)
                zero_mass = f.data.mass.max()/(sorption_list[material]['isotherm'].RH_to_uptake_ads(30)+100)*100
                fig, ax = plt.subplots(1, 1, figsize = (10, 3.5))
                plt.plot(f.data.time, (f.data.mass - zero_mass )/zero_mass*100, label = 'experimental')
                #plt.plot(f.data.time, f.data.RH_target)
                print(data[material][1]['k']['1'])
                print(data[material][1]['k']['0'])
                print(sorption_list[material]['isotherm'].RH_to_uptake_ads(30))
                calculated_curve = curve( sorption_list[material]['isotherm'].RH_to_uptake_ads(30),  
                                            data[material][1]['k']['1']/zero_mass,
                                             data[material][1]['k']['0']/zero_mass,
                                            f.data.time.to_list(), 
                                            f.data.RH_actual.tolist()
                                           )
                plt.plot(f.data.time, calculated_curve, label = 'predicted')
                plt.xlabel('time, min')
                plt.ylabel('Uptake, wt.%/min')
                plt.legend()
                
                kin = analyze_kinetics.Kinetics()
                kin.decompose_to_cycles(f)
                iso = sorption_list[material]['isotherm']
                def working_capacity_exp(Sorption, Isotherm):
                    zero_mass = Sorption.data[Sorption.data.cycle_number == 1].mass.max()/(Isotherm.RH_to_uptake_ads(30)+100)*100
                    print(zero_mass)
                    wc_list = []
                    for cycle in Sorption.data[Sorption.data.cycle_number>0].cycle_number.unique():
                        try:
                            t_ads = Sorption.data[Sorption.data.cycle_number==cycle].time.max()-Sorption.data[Sorption.data.cycle_number==cycle].time.min()
                            t_des = Sorption.data[Sorption.data.cycle_number==-cycle].time.max()-Sorption.data[Sorption.data.cycle_number==-cycle].time.min()
                            #mass_in_cycle = Sorption.data[Sorption.data.cycle_number==cycle].mass.max()-Sorption.data[Sorption.data.cycle_number==-cycle].mass.min()
                            #calculated_mass_in_cycle = max(calculated_curve[Sorption.data[Sorption.data.cycle_number==cycle].index.min():Sorption.data[Sorption.data.cycle_number==cycle].index.max()])\
                            #- min(calculated_curve[Sorption.data[Sorption.data.cycle_number==-cycle].index.min():Sorption.data[Sorption.data.cycle_number==-cycle].index.max()])
                            print(Sorption.data[Sorption.data.cycle_number==cycle].mass.to_list()[-1])
                            mass_in_cycle = Sorption.data[Sorption.data.cycle_number==cycle].mass.to_list()[-1]-Sorption.data[Sorption.data.cycle_number==-cycle].mass.to_list()[-1]
                            calculated_mass_in_cycle = calculated_curve[Sorption.data[Sorption.data.cycle_number==cycle].index.min():Sorption.data[Sorption.data.cycle_number==cycle].index.max()][-1]\
                            - calculated_curve[Sorption.data[Sorption.data.cycle_number==-cycle].index.min():Sorption.data[Sorption.data.cycle_number==-cycle].index.max()][-1]
                            wc = mass_in_cycle/zero_mass/(t_ads + t_des)*100 ## wt.% per min
                            calculated_wc = calculated_mass_in_cycle/(t_ads + t_des)
                            wc_list.append([wc, t_ads, t_des, calculated_wc, zero_mass])
                        except: pass
                    return wc_list
                sorption_list[material]['HM_validation'][f.filename] = working_capacity_exp(f, iso)
                print(sorption_list[material]['HM_validation'][f.filename])

                plt.title('{0}, ads {1} min - des {2} min, sample mass {3} mg'.format(material, 
                                        str(int(round(sorption_list[material]['HM_validation'][f.filename][-2][1], 0))),
                                        str(int(round(sorption_list[material]['HM_validation'][f.filename][-2][2], 0))),
                                        str(int(round(sorption_list[material]['HM_validation'][f.filename][-2][4], 0)))
                                        ))
                plt.savefig('Validation_{0}_ads{1}_des{2}_{3}mg_{4}.png'.format(material, 
                                        str(int(round(sorption_list[material]['HM_validation'][f.filename][-2][1], 0))),
                                        str(int(round(sorption_list[material]['HM_validation'][f.filename][-2][2], 0))),
                                        str(int(round(sorption_list[material]['HM_validation'][f.filename][-2][4], 0))),
                                        str(val_nr)                                        
                                        ))
                plt.show()
    for material in path_dictionary.keys():
        df = pd.DataFrame()
        for sorption in sorption_list[material]['HM_validation'].keys():
            df = df.append(pd.DataFrame(data={'cycle':[''.join([str(int(round(sorption_list[material]['HM_validation'][sorption][-2][1], 0))),
                                 ' min ',
                                 str(int(round(sorption_list[material]['HM_validation'][sorption][-2][2], 0))),
                                 ' min',
                                 ])],
                                 'experimental':[sorption_list[material]['HM_validation'][sorption][-2][0]],
                                 'predicted':[sorption_list[material]['HM_validation'][sorption][-2][3]],
                                'mass':str(int(round(sorption_list[material]['HM_validation'][sorption][-2][4], 0)))
                                  }))
            
            print(sorption_list[material]['HM_validation'][sorption][-2][0])
        for unique_sample_mass in df.mass.unique():
            df[df.mass == unique_sample_mass].plot(x='cycle', y = ['experimental', 'predicted'], kind='bar')
            plt.xticks(rotation = 45)
            plt.xlabel('ads/des times, min')
            plt.ylabel('working capacity per time, wt.%/min')
            plt.title(material + ', ' + unique_sample_mass + 'mg')
            plt.show()